# Voldemort's Stable Diffusion WebUI

Adapted from: https://colab.research.google.com/drive/1Iy-xW9t1-OQWhb0hNxueGij8phCyluOh

**Updating the Notebook**

In the Tools section at the bottom of the notebook there is a block to download the latest version from Github.

**Guides**
- [Getting started on Paperspace](https://github.com/Engineer-of-Stuff/stable-diffusion-paperspace/blob/main/Docs/Paperspace%20Guide%20for%20Retards.md)
- [Using the WebUI](https://rentry.org/voldy)
- [Using the Inpainter](https://rentry.org/drfar)
- [Textual Inversion](https://rentry.org/aikgx)
- [Crowd-Sourced Prompts](https://lexica.art/)
- [Artist Name Prompts](https://sgreens.notion.site/sgreens/4ca6f4e229e24da6845b6d49e6b08ae7?v=fdf861d1c65d456e98904fe3f3670bd3)

## Installation and Setup

You must reinstall everything each time you restart the machine. If already downloaded dependencies will be auto-updated.

**Where to store the models**

`/storage/` is persistent storage shared across all machines on your account.

`/notebooks/` is storage for this notebook only.

We're going to store models in `/storage/models` and create a symlink.

_You must run the block below or else the variable won't be accessable in the notebook._

In [2]:
model_storage_dir = '/storage/models' # no trailing slash

# ===============================================================
# Save the variable to Jupiter's temp storage to access it when the kernel restarts.
# To reset your storage directory, rerun this cell.
%store model_storage_dir

Stored 'model_storage_dir' (str)


### Clone the central repository

In [6]:
import os.path
%store -r model_storage_dir
if not os.path.exists('/notebooks/stable-diffusion-webui'):
    %cd /notebooks/
    !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui
else: # update repo if already exists
    print('stable-diffusion-webui already downloaded, updating...')
    %cd /notebooks/stable-diffusion-webui
    !git pull
    %cd /notebooks/stable-diffusion-webui/scripts
    !wget https://raw.githubusercontent.com/jtkelm2/stable-diffusion-webui-1/master/scripts/wildcards.py
    !wget https://raw.githubusercontent.com/ThereforeGames/txt2img2img/main/scripts/txt2img2img.py
    !wget https://raw.githubusercontent.com/yownas/seed_travel/main/scripts/seed_travel.py
    !wget https://raw.githubusercontent.com/Pfaeff/sd-web-ui-scripts/main/moisaic.py
if not os.path.exists('/notebooks/stable-diffusion-webui/scripts/wildcards/'):
    !mkdir -p /notebooks/stable-diffusion-webui/scripts/wildcards
    %cd /notebooks/stable-diffusion-webui/scripts/wildcards
    !wget https://raw.githubusercontent.com/jtkelm2/stable-diffusion-webui-1/master/scripts/wildcards/adjective.txt
    !wget https://raw.githubusercontent.com/jtkelm2/stable-diffusion-webui-1/master/scripts/wildcards/artist.txt
    !wget https://raw.githubusercontent.com/jtkelm2/stable-diffusion-webui-1/master/scripts/wildcards/genre.txt
    !wget https://raw.githubusercontent.com/jtkelm2/stable-diffusion-webui-1/master/scripts/wildcards/site.txt
    !wget https://raw.githubusercontent.com/jtkelm2/stable-diffusion-webui-1/master/scripts/wildcards/style.txt

stable-diffusion-webui already downloaded, updating...
/notebooks/stable-diffusion-webui
Already up to date.
/notebooks/stable-diffusion-webui/scripts
--2022-09-30 23:22:02--  https://raw.githubusercontent.com/jtkelm2/stable-diffusion-webui-1/master/scripts/wildcards.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1757 (1.7K) [text/plain]
Saving to: ‘wildcards.py’

wildcards.py        100%[===================>]   1.72K  --.-KB/s    in 0s      

2022-09-30 23:22:02 (43.1 MB/s) - ‘wildcards.py’ saved [1757/1757]

--2022-09-30 23:22:03--  https://raw.githubusercontent.com/ThereforeGames/txt2img2img/main/scripts/txt2img2img.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting t

### Install requirements and download repositories

In [7]:
%cd /notebooks/stable-diffusion-webui
%store -r model_storage_dir

# Import launch.py which will automatically run the install script but not launch the WebUI.
# They require a few specific external git repo commits so we have to do it their way. 
import launch

# Download the GFPGAN face restorer.
if not os.path.exists('/notebooks/stable-diffusion-webui/GFPGANv1.3.pth'):
    !wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -O /notebooks/stable-diffusion-webui/GFPGANv1.3.pth
else:
    print('GFPGANv1.3.pth already downloaded')
    
# Make sure your models storage directory exists
!mkdir -p $model_storage_dir

/notebooks/stable-diffusion-webui
GFPGANv1.3.pth already downloaded


### Download the Model

I've provided a few different ways of aquiring the models. Try the torrent option first.

You don't need to repeat this step if you've already downloaded the models.

**Filesize and Storage Disclaimer**

Paperspace free tier has only 5GB of storage space. The standard model is 4GB and waifu is 3.5GB/7.2GB. To get around this limited storage capacity we will download the models to `/storage/`, a non-persistent directory. Each time you restart the machine you will have to redownload the models.

If you don't want to have to redownload the models every time you restart your machine you have three options:
1. Add a payment method to your account. Storage overages are billed at \$0.29/GB and billing occurs monthly and runs at midnight on the first of each month. With a payment method on file, Paperspace will let you use more storage and if you time it right you shouldn't actually be charged for it.
2. Upgrade to a Pro account. They'll give you 15GB and you'll get longer runtimes and more powerful free GPUs.
3. Use my referral code `3E5P612` You'll get \$10 credit that you should be able to put towards the storage overage charges. Redeem the code at the bottom of the Billing page.

**Torrent Instructions**

Aria2 may show some errors/warnings while downloading. Those are fine, when it eventually says "Download Complete" that means everything worked as it should.

#### Standard Model

**Torrent**

In [4]:
%store -r model_storage_dir
!apt update
!apt install -y aria2
%cd $model_storage_dir
!aria2c --seed-time=0 --max-overall-upload-limit=1K "magnet:?xt=urn:btih:3A4A612D75ED088EA542ACAC52F9F45987488D1C&tr=udp://tracker.opentrackr.org:1337/announce"

Get:1 https://deb.nodesource.com/node_16.x focal InRelease [4583 B]
Get:2 https://deb.nodesource.com/node_16.x focal/main amd64 Packages [772 B]   
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]      
Get:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]                
Get:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal/main amd64 Packages [26.6 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [1552 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]        
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal/main amd64 Packages [1275 kB]m
Get:11 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 Packages [27.5 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [906 kB]
Get:13 http://securit

**Web Download**

Voldy provided an alternative download if you don't want to use HuggingFace.

[https://drive.google.com/file/d/1wHFgl0ivCmIZv88hVZXkb8oy9qCuaBGA/view](https://drive.google.com/file/d/1wHFgl0ivCmIZv88hVZXkb8oy9qCuaBGA/view)

Download it to your computer then upload it to your model storage directory (make sure it's named `sd-v1-4.ckpt`).

HuggingFace is much faster and reliable but you need to get access to the repo and provide your user token.

In [ ]:
user_token = "<enter your user token here>"

# ===============================================================================================
%store -r model_storage_dir
!wget --header="'Authorization: Bearer {user_token}'" https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt -O $model_storage_dir/sd-v1-4.ckpt

#### Waifu Diffusion

The original Stable Diffusion anime finetune.

In [5]:
!apt update
!apt install -y aria2
%cd $model_storage_dir
!aria2c --seed-time=0 --max-overall-upload-limit=1K "magnet:?xt=urn:btih:153590FD7E93EE11D8DB951451056C362E3A9150&dn=wd-v1-2-full-ema-pruned.ckpt&tr=udp://tracker.opentrackr.org:1337/announce"

Get:1 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]        
Get:3 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]      
Get:4 https://deb.nodesource.com/node_16.x focal InRelease [4583 B]            
Get:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]0m
Get:6 http://archive.ubuntu.com/ubuntu focal/multiverse amd64 Packages [177 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]      
Get:8 http://archive.ubuntu.com/ubuntu focal/restricted amd64 Packages [33.4 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal/universe amd64 Packages [11.3 MB] 
Get:10 http://archive.ubuntu.com/ubuntu focal/main amd64 Packages [1275 kB]33m 
Get:11 https://deb.nodesource.com/node_16.x focal/main amd64 Packages [772 B]  
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1203 kB]m
Get:13 http://archive.ubuntu.com/ubuntu focal

#### trinart_stable_diffusion_v2

Anime finetune.

The 60000 steps version is the original, the 115000 steps is the 60000 with additional training. Use the 60000 step version if the style nudging is too much.

[See the comparison here](https://raw.githubusercontent.com/Engineer-of-Stuff/stable-diffusion-paperspace/main/Docs/Assets/model%20comparison.jpg)

**60000**

In [ ]:
%store -r model_storage_dir
!wget https://huggingface.co/naclbit/trinart_stable_diffusion_v2/resolve/main/trinart2_step60000.ckpt -O $model_storage_dir/trinart2_step60000.ckpt

**95000**

In [ ]:
%store -r model_storage_dir
!wget https://huggingface.co/naclbit/trinart_stable_diffusion_v2/resolve/main/trinart2_step95000.ckpt -O $model_storage_dir/trinart2_step95000.ckpt

**115000**

In [ ]:
%store -r model_storage_dir
!wget https://huggingface.co/naclbit/trinart_stable_diffusion_v2/resolve/main/trinart2_step115000.ckpt -O $model_storage_dir/trinart2_step115000.ckpt

### Clean up and restart the kernel

In [ ]:
%store -r model_storage_dir

# Get some storage back
!pip cache purge
!rm $model_storage_dir/*.aria2
!apt remove --purge -y aria2
!apt autoremove --purge -y
!apt clean

# Restart the kernel
import os
os.kill(os.getpid(), 9)

Files removed: 142
rm: cannot remove '/storage/models/*.aria2': No such file or directory
Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package aria2
Reading package lists... Done
Building dependency tree       
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


### Link the models directory

Create symlinks. The file will be stored in the models storage directory and linked to where the WebUI expects the files to be.

In [1]:
%store -r model_storage_dir
import os

# Check for broken symlinks and remove them
for file in os.listdir('/notebooks/stable-diffusion-webui/models/'):
    path = f'/notebooks/stable-diffusion-webui/models/{file}'
    if os.path.islink(path) and not os.path.exists(os.readlink(path)):
        print(f'Symlink broken, removing: {file}')
        os.unlink(path)

# Make symlinks for new files
for file in os.listdir(model_storage_dir):
    if file.endswith("ckpt"):
        path = f'/notebooks/stable-diffusion-webui/models/{file}'
        if not os.path.exists(path):
            print(f'New model: {file}')
            !ln -s $model_storage_dir/$file /notebooks/stable-diffusion-webui/models/$file
        !ls -la --block-size=GB /notebooks/stable-diffusion-webui/models/$file

lrwxrwxrwx 1 root root 1GB Sep 29 09:57 /notebooks/stable-diffusion-webui/models/sd-v1-4.ckpt -> /storage/models/sd-v1-4.ckpt
lrwxrwxrwx 1 root root 1GB Sep 29 09:57 /notebooks/stable-diffusion-webui/models/trinart2_step115000.ckpt -> /storage/models/trinart2_step115000.ckpt
lrwxrwxrwx 1 root root 1GB Sep 29 09:57 /notebooks/stable-diffusion-webui/models/wd-v1-2-full-ema-pruned.ckpt -> /storage/models/wd-v1-2-full-ema-pruned.ckpt


# Launch the WebUI

Run this block to launch the WebUI. You will get a link to nnn.gradio.app, that's your WebUI. Follow it.

If you have any issues, restart the kernel.

If you have a lot of VRAM and aren't generating large images you can add the flag `--disable-opt-split-attention` to disable VRAM optimizations for a speed boost.

In [ ]:
%store -r model_storage_dir
%cd /notebooks/stable-diffusion-webui
!python /notebooks/stable-diffusion-webui/webui.py --share #--disable-opt-split-attention

/notebooks/stable-diffusion-webui
Moving sd-v1-4.ckpt from /notebooks/stable-diffusion-webui/models to /notebooks/stable-diffusion-webui/models/Stable-diffusion.
Moving trinart2_step115000.ckpt from /notebooks/stable-diffusion-webui/models to /notebooks/stable-diffusion-webui/models/Stable-diffusion.
Moving wd-v1-2-full-ema-pruned.ckpt from /notebooks/stable-diffusion-webui/models to /notebooks/stable-diffusion-webui/models/Stable-diffusion.
Error loading script: txt2img2img.py
Traceback (most recent call last):
  File "/notebooks/stable-diffusion-webui/modules/scripts.py", line 71, in load_scripts
    exec(compiled, module.__dict__)
  File "/notebooks/stable-diffusion-webui/scripts/txt2img2img.py", line 21, in <module>
    from txt2img2img.dependencies import shortcodes
ModuleNotFoundError: No module named 'txt2img2img'

LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape 

# Export Generations

This block will rename and compress the outputs with 7zip max compression. It expects you to have `log/` and `outputs/` in `/notebooks/`.

In [ ]:
!apt update
!apt install -y p7zip-full
from datetime import datetime
import os
datetime_str = datetime.now().strftime('%m-%d-%Y_%H:%M:%S')
%cd /notebooks/
!mkdir $datetime_str
!mv log $datetime_str
!mv stable-diffusion-webui/outputs $datetime_str
print('Compressing, please wait...')
os.system(f'7z a -t7z -m0=lzma2 -mx=9 -mfb=64 -md=32m -ms=on {datetime_str}.7z {datetime_str}/')

Get:1 https://deb.nodesource.com/node_16.x focal InRelease [4583 B]
Get:2 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]                
Get:3 https://deb.nodesource.com/node_16.x focal/main amd64 Packages [772 B]  m
Get:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]m
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]      
Get:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal/main amd64 Packages [26.6 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [1552 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal/universe amd64 Packages [11.3 MB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 Packages [27.5 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [910 kB]
Get:13 http://security.ub

### Delete old output folder

This block will delete the folder you just compressed.

In [3]:
!rm -rf /notebooks/$datetime_str/
!echo Deleted /notebooks/$datetime_str/

rm: cannot remove '/notebooks/': Device or resource busy
Deleted /notebooks//


# Tools

**Show graphics card info**

In [ ]:
!nvidia-smi

**Download the latest version of this notebook from Github**

Run this and refresh the page (press F5). Don't save anything or you will overwrite the downloaded file.

In [1]:
!mv /notebooks/StableDiffusionUI_Voldemort_paperspace.ipynb /notebooks/StableDiffusionUI_Voldemort_paperspace.ipynb.backup
!wget https://raw.githubusercontent.com/Engineer-of-Stuff/stable-diffusion-paperspace/main/StableDiffusionUI_Voldemort_paperspace.ipynb -O /notebooks/StableDiffusionUI_Voldemort_paperspace.ipynb

--2022-09-30 23:06:54--  https://raw.githubusercontent.com/Engineer-of-Stuff/stable-diffusion-paperspace/main/StableDiffusionUI_Voldemort_paperspace.ipynb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19395 (19K) [text/plain]
Saving to: ‘/notebooks/StableDiffusionUI_Voldemort_paperspace.ipynb’

/notebooks/StableDi 100%[===================>]  18.94K  --.-KB/s    in 0s      

2022-09-30 23:06:54 (124 MB/s) - ‘/notebooks/StableDiffusionUI_Voldemort_paperspace.ipynb’ saved [19395/19395]

